## Output Parsers

In [1]:
!pip install langchain

  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Successfully uninstalled tenacity-9.0.0


In [3]:
from langchain.chains import LLMChain
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [10]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o")
template = """
Specify tags and the main topic of the text.
tags: What are best tags describing text. Give maximum 5 tags separated by comma.
topic: What is the topic of text. Use maximum couple of words

Format response as JSON as below:
'tags': ['sometag', 'othertag', 'anothertag', 'tag4', 'tag5']
'subject': 'Some subject of text'

text: {input}
"""

prompt_template = ChatPromptTemplate.from_template(template=template)
chain = LLMChain(llm=llm, prompt=prompt_template)
chain.predict(input="They picked a way among the trees, and their ponies plodded along, carefully avoiding the many writhing and interlacing roots.  There was no undergrowth.  The ground was rising steadily, and as they went forward it seemed that the trees became taller, darker, and thicker. There was no sound, except an occasional drip of moisture falling through the still leaves.  For the moment there was no whispering or movement among the branches; but they all got an uncomfortable feeling that they were being watched with disapproval, deepening to dislike and even enmity.  The feeling steadily grew, until they found themselves looking up quickly, or glancing back over their shoulders, as if they expected a sudden blow.")

'```json\n{\n  "tags": ["forest", "ponies", "suspense", "nature", "trees"],\n  "topic": "forest journey"\n}\n```'

## ResponseSchema and OutputParsers 

In [13]:
from langchain.output_parsers import ResponseSchema
from langchain.output_parsers import StructuredOutputParser
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

tags_schema = ResponseSchema(
    name="tags",
    description=" What are best tags describing text. Give maximum 5 tags separated by comma.",
)
topic_schema = ResponseSchema(
    name="topic", 
    description="What is the topic of text. Use maximum couple of words."
)

response_schemas = [tags_schema, topic_schema]
parser = StructuredOutputParser.from_response_schemas(response_schemas)
template = """
Specify tags and the main topic of the text.
tags: What are best tags describing text. Give maximum 5 tags separated by comma.
topic: What is the topic of text. Use maximum couple of words

Format response as JSON as below:
'tags': ['sometag', 'othertag', 'anothertag', 'tag4', 'tag5']
'subject': 'Some subject of text'

text: {input}

{instructions}
"""

prompt = ChatPromptTemplate.from_template(template=template)
instructions = parser.get_format_instructions()

messages = prompt.format_messages(
    input="They picked a way among the trees, and their ponies plodded along, carefully avoiding the many writhing and interlacing roots.  There was no undergrowth.  The ground was rising steadily, and as they went forward it seemed that the trees became taller, darker, and thicker. There was no sound, except an occasional drip of moisture falling through the still leaves.  For the moment there was no whispering or movement among the branches; but they all got an uncomfortable feeling that they were being watched with disapproval, deepening to dislike and even enmity.  The feeling steadily grew, until they found themselves looking up quickly, or glancing back over their shoulders, as if they expected a sudden blow.",
    instructions=instructions,
)

chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response)
output_dict = parser.parse(response.content)
print(output_dict)

content='```json\n{\n\t"tags": ["fantasy, adventure, suspense, nature, mystery"],\n\t"topic": "Journey through the forest"\n}\n```' response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 325, 'total_tokens': 358, 'completion_tokens_details': {'reasoning_tokens': 0}}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-6a93a882-bbd2-43de-b6e0-c2c69760abcf-0'
{'tags': ['fantasy, adventure, suspense, nature, mystery'], 'topic': 'Journey through the forest'}
